# Basics
このノートでは、pysparkを用いてデータフレームを基本的な操作する例を示した。

## SparkSessionを立てる
pythonからpysparkを使う時には、SparkSession.builder.getOrCreate()でセッションを立てる。
ローカルで実行する際にはこれでとりあえず使える。

## リソース管理
このままだと全てのCPU coreを使うので、制限したい場合はmaster("local[4]")のように指定する。
config("spark.driver.memory","2g")のようにセッションの設定値を指定する。

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
#spark = SparkSession.builder.master("local[4]") \
#                            .config("spark.driver.memory","2g") \
#                            .config("spark.executor.memory","2g") \
#                            .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/08 17:14:03 WARN Utils: Your hostname, TABLET-S9I8ER9S, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/08 17:14:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/08 17:14:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Parquetファイルを開く
- spark.read.parquet(): parquetファイルを開いてデータフレームを作成する。
- df: データフレーム
- df.show(): データフレームを表示。引数で行数指定。

In [2]:
df = spark.read.parquet("./data/clover_data.parquet")
df.show(10, truncate=False)

+-------------+---+-----+----+
|ts           |ch |cfd  |adc |
+-------------+---+-----+----+
|7296321907778|4  |8485 |3982|
|7296321978243|2  |9278 |774 |
|7296321997358|1  |8225 |1760|
|7296322069363|0  |16234|0   |
|7296322214400|4  |24255|4526|
|7296322271040|1  |32358|751 |
|7296322391382|4  |26343|1315|
|7296322439661|0  |21550|0   |
|7296322447462|7  |3413 |245 |
|7296322574024|2  |12205|3798|
+-------------+---+-----+----+
only showing top 10 rows


## Schemaの表示
printSchema()でデータフレームの構造を表示させることができる。

In [3]:
df.printSchema()

root
 |-- ts: long (nullable = true)
 |-- ch: short (nullable = true)
 |-- cfd: short (nullable = true)
 |-- adc: integer (nullable = true)



## その他関数
- count(): 行数をカウントする。

In [4]:
df.count()

521829

- select(): 指定した列だけ取り出す。

In [5]:
df = df.select("ch","adc","ts")
df.show(5)

+---+----+-------------+
| ch| adc|           ts|
+---+----+-------------+
|  4|3982|7296321907778|
|  2| 774|7296321978243|
|  1|1760|7296321997358|
|  0|   0|7296322069363|
|  4|4526|7296322214400|
+---+----+-------------+
only showing top 5 rows


- filter(): 条件を満たす行だけ取り出す。

In [6]:
df = df.filter("ch>=0 AND ch<4")
df.show(5)

+---+----+-------------+
| ch| adc|           ts|
+---+----+-------------+
|  2| 774|7296321978243|
|  1|1760|7296321997358|
|  0|   0|7296322069363|
|  1| 751|7296322271040|
|  0|   0|7296322439661|
+---+----+-------------+
only showing top 5 rows


## spark.sql.functions

様々なpyspark SQL関数が定義されている。以降、`F`としてインポートしておく
- F.col(): データフレームの列を返す
- F.lit(): 定数からなる列を返す
- withColumn(): 列を追加

In [7]:
from pyspark.sql import functions as F
# ts (ns)をsecに変換した列を定義
df.withColumn("ts_in_sec", F.lit(1.e-9)*F.col("ts")).show(5)

+---+----+-------------+------------------+
| ch| adc|           ts|         ts_in_sec|
+---+----+-------------+------------------+
|  2| 774|7296321978243| 7296.321978243001|
|  1|1760|7296321997358| 7296.321997358001|
|  0|   0|7296322069363|    7296.322069363|
|  1| 751|7296322271040|7296.3222710400005|
|  0|   0|7296322439661|    7296.322439661|
+---+----+-------------+------------------+
only showing top 5 rows


- F.expr(): データフレームの列に対してSQL表記の操作を行う。

上のセルのような操作を完結に表記できる。

In [8]:
df.withColumn("ts_in_sec", F.expr("1.e-9*ts")).show(5)

+---+----+-------------+------------------+
| ch| adc|           ts|         ts_in_sec|
+---+----+-------------+------------------+
|  2| 774|7296321978243| 7296.321978243001|
|  1|1760|7296321997358| 7296.321997358001|
|  0|   0|7296322069363|    7296.322069363|
|  1| 751|7296322271040|7296.3222710400005|
|  0|   0|7296322439661|    7296.322439661|
+---+----+-------------+------------------+
only showing top 5 rows


- spark.sql(): SQLコマンドを実行する。
- df.createOrReplaceTempView(): データフレームをSQLで呼ぶためのTempViewに登録する。

下はSQLで"ch"毎の"adc"の値の平均をとる操作を行うもの。

In [9]:
df.createOrReplaceTempView("df")
result_df = spark.sql("""
    SELECT ch, AVG(adc) AS avg_adc
    FROM df
    GROUP BY ch
""")
result_df.show()

+---+------------------+
| ch|           avg_adc|
+---+------------------+
|  1|2041.8029956882683|
|  3| 5516.067150776918|
|  2|1937.2271148371594|
|  0| 4341.374239243097|
+---+------------------+

